<h1 align="center">MLB War Predictor</h1>


## Plan

1. Grab player stats for features
2. Add them to a SQL database 
3. Use SQL to make the features ready for ML
4. Pull the features into a pandas dataframe
5. Fill in any missing stats and split into train test splits
6. Train and evaluate the Pytorch model
7. Send the results back into SQL
8. Use the SQL db to make a PowerBI dashboard

#### Add Ons
1. Contract efficiency values for the dashboard (Maybe extend into a contract predictor)
2. Add in an adjustment to WAR using injury prediction


## Note

I am doing only hitters to start as pitchers would require a new framework  
All stats use pybaseball library to grab them from Fangraphs

# Step 1.

In [84]:
import pandas as pd
from pybaseball import batting_stats
import os
import time

Loading the stats from Fangraphs

In [85]:
startingYear = 2021
endingYear = 2025

Stats = batting_stats(startingYear, endingYear)

The next 3 lines of code is to see what this new dataframe's structure looks like and gives more context on the features

In [86]:
Stats.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2024,Aaron Judge,NYY,32,158,559,704,180,85,...,117.5,239,0.611,391,0.146,0.267,0.305,0.740,0.481,11.7
2,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,247,0.611,404,0.169,0.287,0.304,0.732,0.468,11.4
17,25764,2024,Bobby Witt Jr.,KCR,24,161,636,709,211,123,...,116.9,260,0.483,538,0.138,0.236,0.309,0.583,0.407,10.5
1,15640,2025,Aaron Judge,NYY,33,152,541,679,179,94,...,118.1,226,0.582,388,0.150,0.292,0.300,0.708,0.459,10.2
6,18401,2023,Ronald Acuna Jr.,ATL,25,159,643,735,217,137,...,121.2,310,0.552,562,0.137,0.214,0.351,0.668,0.461,8.5


In [87]:
Stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 670 entries, 0 to 573
Columns: 320 entries, IDfg to L-WAR
dtypes: float64(262), int64(54), object(4)
memory usage: 1.6+ MB


In [88]:
Stats.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
count,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,...,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000,670.000000
mean,17335.141791,2023.037313,28.219403,145.776119,538.077612,604.080597,140.392537,88.398507,27.973134,2.279104,...,112.000746,174.549254,0.418800,419.167164,0.161076,0.265636,0.254587,0.443251,0.335372,2.954925
std,6283.741872,1.428970,3.566735,10.649492,53.990670,61.029700,22.105933,18.432708,6.735169,2.279698,...,2.904182,35.634624,0.074541,57.090300,0.031039,0.024924,0.021737,0.066025,0.032151,1.969323
min,1744.000000,2021.000000,20.000000,116.000000,419.000000,502.000000,83.000000,44.000000,9.000000,0.000000,...,103.200000,65.000000,0.149000,281.000000,0.075000,0.186000,0.186000,0.302000,0.266000,-1.900000
25%,13145.000000,2022.000000,26.000000,138.000000,494.000000,550.000000,124.000000,76.000000,23.000000,1.000000,...,110.000000,150.000000,0.376000,380.000000,0.140000,0.250000,0.241000,0.400250,0.314250,1.600000
50%,17649.000000,2023.000000,28.000000,147.000000,537.500000,601.000000,139.000000,88.000000,28.000000,2.000000,...,111.600000,173.000000,0.423000,414.000000,0.160000,0.266000,0.254000,0.438000,0.332000,2.800000
75%,20381.250000,2024.000000,31.000000,154.000000,581.000000,653.000000,155.000000,100.000000,32.000000,3.000000,...,113.800000,197.000000,0.468000,458.000000,0.183750,0.283000,0.268000,0.475000,0.352000,4.100000
max,33824.000000,2025.000000,41.000000,163.000000,671.000000,753.000000,217.000000,161.000000,59.000000,17.000000,...,122.900000,310.000000,0.611000,618.000000,0.255000,0.329000,0.351000,0.740000,0.481000,11.700000


In [89]:
list(Stats.columns)

['IDfg',
 'Season',
 'Name',
 'Team',
 'Age',
 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'GDP',
 'SB',
 'CS',
 'AVG',
 'GB',
 'FB',
 'LD',
 'IFFB',
 'Pitches',
 'Balls',
 'Strikes',
 'IFH',
 'BU',
 'BUH',
 'BB%',
 'K%',
 'BB/K',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'wOBA',
 'wRAA',
 'wRC',
 'Bat',
 'Fld',
 'Rep',
 'Pos',
 'RAR',
 'WAR',
 'Dol',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'phLI',
 'PH',
 'WPA/LI',
 'Clutch',
 'FB% (Pitch)',
 'FBv',
 'SL%',
 'SLv',
 'CT%',
 'CTv',
 'CB%',
 'CBv',
 'CH%',
 'CHv',
 'SF%',
 'SFv',
 'KN%',
 'KNv',
 'XX%',
 'PO%',
 'wFB',
 'wSL',
 'wCT',
 'wCB',
 'wCH',
 'wSF',
 'wKN',
 'wFB/C',
 'wSL/C',
 'wCT/C',
 'wCB/C',
 'wCH/C',
 'wSF/C',
 'wKN/C',
 'O-Swing%',
 'Z-Swing%',
 'Swing%',
 'O-Contact%',
 'Z-Contact%',
 'Contact%',
 'Zone%',
 'F-Strike%',
 'SwStr%',
 'BsR',
 'FA% (sc)',
 

Select all the columns we want to keep for the model and dashboard

In [90]:
Stats = Stats[['IDfg', 'Name', 'Season', 'Team', 'G', 'Age', 'PA', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'AVG', 'OBP', 'SLG', 'OPS', 'BB/K', 'LD%', 'GB%', 'FB%', 'HR/FB', 'wRC+', 'Pos', 'Fld', 'WAR', 'Spd', 'WPA', 'Clutch', 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%', 'EV', 'LA', 'Barrel%', 'xwOBA']]

Just doing some quick cleaning to ensure that all the batters are qualified, as non qualified batters have less data, and their data is less consistent with their true performance.    
Qualified hitters are defined by MLB as having 3.1 PA per scheduled game, as the season has 162 scheduled games we will set the cuttoff for qualification at 502.2 PA.

In [91]:
Stats = Stats[Stats['PA'] >= 503]

Stats.dropna(subset=['WAR'])

,IDfg,Name,Season,Team,G,Age,PA,R,H,2B,...,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,EV,LA,Barrel%,xwOBA
0,15640,Aaron Judge,2024,NYY,158,32,704,122,180,36,...,0.400,0.349,0.251,0.072,0.451,0.477,96.2,18.9,0.269,0.481
2,15640,Aaron Judge,2022,NYY,157,30,696,133,177,28,...,0.475,0.318,0.208,0.075,0.445,0.480,95.8,14.9,0.262,0.468
17,25764,Bobby Witt Jr.,2024,KCR,161,24,709,125,211,45,...,0.351,0.385,0.264,0.147,0.500,0.353,92.7,15.1,0.143,0.407
1,15640,Aaron Judge,2025,NYY,152,33,679,137,179,30,...,0.366,0.374,0.260,0.098,0.446,0.456,95.4,19.1,0.247,0.459
6,18401,Ronald Acuna Jr.,2023,ATL,159,25,735,149,217,35,...,0.397,0.356,0.247,0.101,0.438,0.461,94.7,7.4,0.153,0.461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,15117,Hunter Dozier,2021,KCR,144,29,543,55,105,27,...,0.416,0.372,0.212,0.124,0.510,0.366,89.9,15.8,0.088,0.298
652,21897,Christopher Morel,2024,- - -,152,25,611,56,105,11,...,0.465,0.365,0.171,0.186,0.478,0.336,89.2,14.2,0.102,0.316
646,19600,Bryan De La Cruz,2024,- - -,149,27,622,61,136,25,...,0.420,0.308,0.272,0.126,0.547,0.327,89.6,13.1,0.093,0.306
649,10472,Enrique Hernandez,2023,- - -,140,31,508,57,110,23,...,0.366,0.374,0.259,0.174,0.561,0.265,87.8,17.1,0.045,0.272


We can see that some players have their team value as - - -, thorugh further research it was found that this occurs when a player has more that 1 team. Thus we will fill these with an abbreviation to indicate that they had multiple teams

In [92]:
Stats['Team'] = Stats['Team'].replace('- - -', 'MLT')

In [93]:
Stats[Stats['Name'] == 'Christopher Morel']

,IDfg,Name,Season,Team,G,Age,PA,R,H,2B,...,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,EV,LA,Barrel%,xwOBA
652,21897,Christopher Morel,2024,MLT,152,25,611,56,105,11,...,0.465,0.365,0.171,0.186,0.478,0.336,89.2,14.2,0.102,0.316


# Step 2.

Now that the relevant data has been pulled into a dataframe, we can then send the data into our SQl server 

In [94]:
# First we rename the columns to more descriptive names for the SQL database

Stats.rename(columns={'IDfg': 'PlayerID', 'PA': 'Plate Appearances', 'G': 'Games Played', 'R': 'Runs', 'H': 'Hits', '2B': 'Doubles', '3B': 'Triples', 'HR': 'Home Runs', 'RBI': 'Runs Batted In', 'SB': 'Stolen Bases', 'CS': 'Caught Stealing', 'BB': 'Walks', 'SO': 'Strikeouts', 
                      'AVG': 'Batting Average', 'OBP': 'On-base Percentage', 'SLG': 'Slugging Percentage', 'OPS': 'On-base Plus Slugging', 'BB/K': 'Walks to Strikeouts Ratio', 'LD%': 'Line Drive Percentage', 'GB%': 'Ground Ball Percentage', 'FB%': 'Fly Ball Percentage', 'HR/FB': 'Home Run to Fly Ball Ratio', 'wRC+': 'Weighted Runs Created Plus', 'Pos':'Positional Adjustment','Fld': 'Fielding Runs', 'WAR': 'Wins Above Replacement', 'Spd':'Speed Score', 'WPA': 'Win Probability Added', 'Clutch': 'Clutch Score', 'Pull%': 'Pull Field Hit Percentage',
  
 'Cent%': 'Center Field Hit Percentage', 'Oppo%': 'Opposite Field Hit Percentage', 'Soft%': 'Soft Hit Percentage', 'Med%': 'Medium Hit Percentage', 'Hard%': 'Hard Hit Percentage', 'EV': 'Exit Velocity', 'LA': 'Launch Angle', 'Barrel%': 'Barrel Percentage', 'xwOBA': 'Expected Weighted On-base Average'}, inplace=True)

In [95]:
'''# Now we separate the data into three different dataframes for easier SQL database management by using 3 tables to hold the data

# Isolated the player info as having a quick way to see all the players will be useful
players = Stats[['PlayerID', 'Name']].drop_duplicates()

# Isolated the WAR values as that is the main target variable we are predicting
WARVals = Stats[['PlayerID', 'Season', 'Wins Above Replacement']]

# Isolated the advanced stats for ease of access when creating the dashboards
advStats = Stats[['PlayerID','Season', 'Walks to Strikeouts Ratio', 'Line Drive Percentage', 'Ground Ball Percentage', 'Fly Ball Percentage', 'Home Run to Fly Ball Ratio', 'Weighted Runs Created Plus', 'Positional Adjustment', 'Fielding Runs',  'Speed Score', 'Win Probability Added', 'Clutch Score', 'Pull Field Hit Percentage',
 'Center Field Hit Percentage',
 'Opposite Field Hit Percentage',
 'Soft Hit Percentage',
 'Medium Hit Percentage',
 'Hard Hit Percentage', 'Exit Velocity', 'Launch Angle', 'Barrel Percentage', 'Expected Weighted On-base Average']]

# Finally we wish to keep the main batting stats together
Stats = Stats[['PlayerID', 'Name', 'Season', 'Team', 'Games Played', 'Age', 'Plate Appearances', 'Runs', 'Hits', 'Doubles', 'Triples', 'Home Runs', 'Runs Batted In', 'Stolen Bases', 'Caught Stealing', 'Walks', 'Strikeouts', 'Batting Average', 'On-base Percentage', 'Slugging Percentage', 'On-base Plus Slugging']] 

'''

"# Now we separate the data into three different dataframes for easier SQL database management by using 3 tables to hold the data\n\n# Isolated the player info as having a quick way to see all the players will be useful\nplayers = Stats[['PlayerID', 'Name']].drop_duplicates()\n\n# Isolated the WAR values as that is the main target variable we are predicting\nWARVals = Stats[['PlayerID', 'Season', 'Wins Above Replacement']]\n\n# Isolated the advanced stats for ease of access when creating the dashboards\nadvStats = Stats[['PlayerID','Season', 'Walks to Strikeouts Ratio', 'Line Drive Percentage', 'Ground Ball Percentage', 'Fly Ball Percentage', 'Home Run to Fly Ball Ratio', 'Weighted Runs Created Plus', 'Positional Adjustment', 'Fielding Runs',  'Speed Score', 'Win Probability Added', 'Clutch Score', 'Pull Field Hit Percentage',\n 'Center Field Hit Percentage',\n 'Opposite Field Hit Percentage',\n 'Soft Hit Percentage',\n 'Medium Hit Percentage',\n 'Hard Hit Percentage', 'Exit Velocity',

In [96]:
# We will be using SQLAlchemy and psycopg2 to interface with the PostgreSQL database
from sqlalchemy import create_engine

from dotenv import load_dotenv

# Import the environment variables containing the database connection info for safe access
load_dotenv()

# Create the database engine to connect to the PostgreSQL database
engine = create_engine(f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}")

try:
    connection = engine.connect()
    print("Connection to PostgreSQL established successfully!")
except Exception as e:
    print(f"Connection failed! Error: {e}")

Connection to PostgreSQL established successfully!


In [97]:
'''# Create the tables and insert the data in the PostgreSQL database
players.to_sql('Players', engine, if_exists='fail', index=False, method='multi', chunksize=1000)
WARVals.to_sql('WARVals', engine, if_exists='fail', index=False, method='multi', chunksize=1000)
advStats.to_sql('AdvStats', engine, if_exists='fail', index=False, method='multi', chunksize=1000)
Stats.to_sql('Stats', engine, if_exists='fail', index=False, method='multi', chunksize=1000)'''

"# Create the tables and insert the data in the PostgreSQL database\nplayers.to_sql('Players', engine, if_exists='fail', index=False, method='multi', chunksize=1000)\nWARVals.to_sql('WARVals', engine, if_exists='fail', index=False, method='multi', chunksize=1000)\nadvStats.to_sql('AdvStats', engine, if_exists='fail', index=False, method='multi', chunksize=1000)\nStats.to_sql('Stats', engine, if_exists='fail', index=False, method='multi', chunksize=1000)"

Note that in order to make the insertion fast, these commands must create the tables from scatch, as they are rather quick commmands you may wish to drop an old table if you wish to update, can probably improve in the future to update better however for ease when creating the script it seems better to just brute force it

In [98]:
'''# Run a sanity check to ensure the data was written correctly to Players table
query = 'SELECT * FROM "Players";'

playerData = pd.read_sql(query, connection)
playerData.head()'''

'# Run a sanity check to ensure the data was written correctly to Players table\nquery = \'SELECT * FROM "Players";\'\n\nplayerData = pd.read_sql(query, connection)\nplayerData.head()'

In [99]:
'''# Run a sanity check to ensure the data was written correctly to WARVals table
query = 'SELECT * FROM "WARVals";'

WARData = pd.read_sql(query, connection)
WARData.head()'''

'# Run a sanity check to ensure the data was written correctly to WARVals table\nquery = \'SELECT * FROM "WARVals";\'\n\nWARData = pd.read_sql(query, connection)\nWARData.head()'

In [100]:
'''# Run a sanity check to ensure the data was written correctly to AdvStats table
query = 'SELECT * FROM "AdvStats";'

advStatsData = pd.read_sql(query, connection)
advStatsData.head()'''

'# Run a sanity check to ensure the data was written correctly to AdvStats table\nquery = \'SELECT * FROM "AdvStats";\'\n\nadvStatsData = pd.read_sql(query, connection)\nadvStatsData.head()'

In [101]:
'''# Run a sanity check to ensure the data was written correctly to Stats table
query = 'SELECT * FROM "Stats";'

statsData = pd.read_sql(query, connection)
statsData.head()'''

'# Run a sanity check to ensure the data was written correctly to Stats table\nquery = \'SELECT * FROM "Stats";\'\n\nstatsData = pd.read_sql(query, connection)\nstatsData.head()'

I am now realizing that splitting the data into separate tables seems like its more organized now, however it makes the resulting work from it more complex. Maybe I might try to solve this in the future, however now I am just gonna keep it simple to ensure I can get a working copy ready.

Want to eventually make it an Upsert allowing for updates and adding new data however for now to update I will have to delete the old table.

In [ ]:
# Select float columns
float_cols = Stats.select_dtypes(include='float').columns

# Round only floats to 3 decimals as that is how they are traditionally stored for baseball
Stats[float_cols] = Stats[float_cols].round(3)

# Write all the Stats to the database 
Stats.to_sql('Batter Stats', engine, if_exists='fail', index=False, method='multi', chunksize=1000)

668

In [ ]:
# Run a sanity check to ensure the data was written correctly to Stats table
query = 'SELECT * FROM "Batter Stats";'

Stats = pd.read_sql(query, connection)
Stats.head

<bound method NDFrame.head of      PlayerID               Name  Season Team  Games Played  Age  \
0       15640        Aaron Judge    2024  NYY           158   32   
1       15640        Aaron Judge    2022  NYY           157   30   
2       25764     Bobby Witt Jr.    2024  KCR           161   24   
3       15640        Aaron Judge    2025  NYY           152   33   
4       18401   Ronald Acuna Jr.    2023  ATL           159   25   
..        ...                ...     ...  ...           ...  ...   
663     15117      Hunter Dozier    2021  KCR           144   29   
664     21897  Christopher Morel    2024  MLT           152   25   
665     19600   Bryan De La Cruz    2024  MLT           149   27   
666     10472  Enrique Hernandez    2023  MLT           140   31   
667     10815   Jurickson Profar    2023  MLT           125   30   

     Plate Appearances  Runs  Hits  Doubles  ...  Pull Field Hit Percentage  \
0                  704   122   180       36  ...                      0.40

In [ ]:
# Close the connection to the database
connection.close()

# Step 3.